In [1]:
import re
import plotly
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
import matplotlib
from random import sample
import numpy as np

**Heatmap**

In [ ]:
# Load the CSV data into DataFrames
df_avian = pd.read_csv("gs_gd_avian.csv")
df_swine = pd.read_csv("gs_gd_swine.csv")
df_human = pd.read_csv("gs_gd_human.csv")

freq_list = [["n = 33", "n = 4860", "n = 3982", "n = 8668"],
             ["n = 0", "n = 34", "n = 19", "n = 0"],
             ["n = 19", "n = 542", "n = 207", "n = 26"]]

title = ["Avian", "Swine", "Human"]
for df in [df_avian, df_swine, df_human]:
    df.set_index("Proteins", inplace=True)
    df.round(2)
    df.dropna(inplace=True)

# Function to create mask and annotation DataFrame
def create_mask_annot(df):
    mask = df >= 0.01
    annot_df = df.mask(~mask, "")
    return annot_df

annot_df_avian = create_mask_annot(df_avian)
annot_df_swine = create_mask_annot(df_swine)
annot_df_human = create_mask_annot(df_human)


In [ ]:
sns.set(font_scale=1.2)

# Create the figure and axes
fig, axs = plt.subplots(3, 1, figsize=(8, 8))

# Plot the heatmaps
for i, (ax, df, annot_df) in enumerate(zip(axs, [df_avian, df_swine, df_human], [annot_df_avian, annot_df_swine, annot_df_human])):
    heatmap = sns.heatmap(df, annot=annot_df, fmt="",
                          cmap=sns.cubehelix_palette(dark=.25, light=.85, as_cmap=True),
                          linewidth=0.75, cbar=True, xticklabels=True, yticklabels=True, ax=ax, cbar_kws={'pad': 0.18})

    heatmap.set_yticklabels(heatmap.get_yticklabels(), rotation=0)

    # Add x tick labels only to the first plot
    if i == 0:
        heatmap.tick_params(axis='x', top=True, labeltop=True, bottom=False, labelbottom=False)
    else:
        heatmap.tick_params(axis='x', top=False, labeltop=False, bottom=False, labelbottom=False)

    heatmap.set_xticklabels(heatmap.get_xticklabels(), rotation=90)
    ax.set_ylabel(" ")
    # Create a twin y-axis to add labels on the right
    ax2 = ax.twinx()
    ax2.set_ylim(ax.get_ylim())
    ax2.set_yticks(ax.get_yticks())
    ax2.set_yticklabels(freq_list[i], rotation=0)

    # Hide the right y-axis ticks and label
    ax2.tick_params(axis='y', which='both', length=0)
    ax2.set_ylabel(" ")
    ax2.grid(False)

    # Make sure both axes share the same position
    pos1 = ax.get_position()
    ax2.set_position(pos1)

    for _, spine in heatmap.spines.items():
        spine.set_visible(True)
        spine.set_color('black')
        spine.set_linewidth(1)

# Add titles outside the loop
for i, ax in enumerate(axs):
    ax.set_title(title[i], fontweight="bold")

# Adjust the vertical spacing between subplots
plt.subplots_adjust(hspace=0.4)  # Change 0.4 to your desired spacing

plt.ylabel(" ", rotation=0, labelpad=20)

# Adjust layout to avoid clipping of tick labels
plt.tight_layout()

# Save the figure
plt.savefig('multiple_heatmaps.tiff', dpi = 600, bbox_inches='tight')
plt.show()


**NA Stalk length analysis**

In [ ]:
def clean_msa(input_file,n):

  lines_to_concatenate = n
  with open(input_file, "r") as file:
    lines = file.readlines()
  # Initialize a list to store lines to be appended
  appended_lines = []
  # Iterate through the lines and concatenate lines from each block
  for i in range(lines_to_concatenate):
    for j in range(i, len(lines), lines_to_concatenate):
        appended_lines.append(lines[j].strip())
  # Write the extracted lines into the output file
  with open("output.txt", "w") as file:
    file.write("\n".join(appended_lines))


  lines = []
  with open("output.txt","r") as file:
    lines.append(file.readlines())
  # Initialize variables to store the concatenated strings
  concatenated_strings = []
  current_string = ""
  # Iterate through the list of strings
  for line in lines[0]:
    if '\t' not in line:
      # If there is no '\t' in the line, add it to the current string
      current_string += line.strip()
    else:
      # If a '\t' is found, add it to the current string and append it to the concatenated_strings list
      current_string += line.strip()
      concatenated_strings.append(current_string)
      current_string = ""  # Reset the current string


  return concatenated_strings

In [ ]:
def get_stalk(input_string,stak_begin_regex,stalk_end_regex, A, A_no, B, B_no):

  match = re.search(stak_begin_regex, input_string)
  if match:
    matched_text = match.group()
    start_index = match.start()
    end_index = match.end()
    # Find the index of the second occurrence of "I" within the matched text
    second_i_index = matched_text.find(A, matched_text.find(A) +A_no )
  else:
    print("Stalk beginning not found.")

  start_index_stalk = start_index+second_i_index


  # Search for the pattern in the input string, case-insensitive
  match = re.search(stalk_end_regex, input_string, re.IGNORECASE)

  if match:
    matched_text = match.group()
    start_index_ = match.start()
    end_index_ = match.end()

    # Find the index of the second occurrence of "x" within the matched text
    second_i_index_ = matched_text.find(B, matched_text.find(B) +B_no )
  else:
    print("Stalk end not found.")

  end_index_stalk = start_index_ + second_i_index_

  return [start_index_stalk,end_index_stalk]


In [ ]:
def get_stalk_length_year(input_file_msa, input_msa_date, n, stak_begin_regex ,stalk_end_regex, A, A_no, B, B_no ):

  # Open the input file for reading and the output file for writing
  with open(input_file_msa, 'r') as infile, open("output_"+input_file_msa, 'w') as outfile:
    # Iterate through each line in the input file
    for line in infile:
        # Check if the line is not blank (contains at least one character)
        if line.strip():
            # If the line is not blank, write it to the output file
            outfile.write(line)

  msa = clean_msa("output_"+input_file_msa,n)
  stalk_cs = get_stalk(msa[0],stak_begin_regex ,stalk_end_regex,A, A_no, B, B_no)

  # for every sequence in msa except cs isolate the stalk
  stalk_list = []
  for i in range(1,len(msa)):
    stalk_list.append(msa[i][stalk_cs[0]:stalk_cs[1]])
  # for every stalk get length without counting -
  stalk_length= []

  for input_string in stalk_list:
    # Remove hyphens from the input string
    input_string_without_hyphens = input_string.replace("-", "")

    # Calculate the length of the modified string and append it to the lengths list
    length_without_hyphens = len(input_string_without_hyphens)
    stalk_length.append(length_without_hyphens)

  lines = []
  with open(input_msa_date,"r") as f:
    lines.append(f.readlines())

  regex_year = '\|\d{4}-\d{2}-\d{2}\|'
  years = []
  for i in range(0,len(lines[0])):
    input_string = lines[0][i]
    # Search for the pattern in the input string, case-insensitive
    match = re.search(regex_year, input_string)
    if match:
      matched_text = match.group()

      years.append(int(matched_text.split('-')[0][1:5]))

  lines = []
  with open(input_msa_date,"r") as f:
    lines.append(f.readlines())

  regex_epi =  r'\|EPI_ISL_\d+\|'
  epi = []
  for i in range(0,len(lines[0])):
    input_string = lines[0][i]
  # Search for the pattern in the input string, case-insensitive
    match = re.search(regex_epi, input_string)
    if match:
      matched_text = match.group()
      epi.append(matched_text)

  ha_na_regex = '\|A_/_H\d{1,2}N\d{1,2}\|'
  HA = []
  for i in range(0,len(lines[0])):
    input_string = lines[0][i]
  # Search for the pattern in the input string, case-insensitive
    match = re.search(ha_na_regex, input_string)
    if match:
      matched_text = match.group()
      HA.append(matched_text)

  return years, msa[1:], stalk_list,stalk_length, epi, HA

In [ ]:
# Avian stalk length
years_batch1,full_sequences_1, stalk_list_1, stalk_length_batch1, epi_batch1, _ =  get_stalk_length_year("N1-msa-batch1.txt", "N1-msa-date-batch1.txt", 4000, r'G[-]*N[-]*I[-]*I[-]*S[-]*x[-]*x', r'V[-]*x[-]*x[-]*x[-]*T[-]*L', "I", 1, "x", 1)
years_batch2,full_sequences_2, stalk_list_2, stalk_length_batch2, epi_batch2, _  =  get_stalk_length_year("N1-msa-batch2.txt", "N1-msa-date-batch2.txt", 4000, r'G[-]*N[-]*I[-]*I[-]*S[-]*x[-]*x', r'V[-]*x[-]*x[-]*x[-]*T[-]*L', "I", 1, "x", 1)
years_batch3,full_sequences_3, stalk_list_3, stalk_length_batch3, epi_batch3, _  =  get_stalk_length_year("N1-msa-batch3.txt", "N1-msa-date-batch3.txt", 4000, r'G[-]*N[-]*I[-]*I[-]*S[-]*x[-]*x', r'V[-]*x[-]*x[-]*x[-]*T[-]*L', "I", 1, "x", 1)
years_batch4,full_sequences_4, stalk_list_4, stalk_length_batch4, epi_batch4, _  =  get_stalk_length_year("N1-msa-batch4.txt", "N1-msa-date-batch4.txt", 2322, r'G[-]*N[-]*I[-]*I[-]*S[-]*x[-]*x', r'V[-]*x[-]*x[-]*x[-]*T[-]*L', "I", 1, "x", 1)

years = years_batch1 + years_batch2 + years_batch3 + years_batch4
full_sequences = full_sequences_1+full_sequences_2+full_sequences_3 + full_sequences_4
stalk_list = stalk_list_1 + stalk_list_2 + stalk_list_3 + stalk_list_4
stalk_length = stalk_length_batch1 + stalk_length_batch2+ stalk_length_batch3 + stalk_length_batch4
epi = epi_batch1 + epi_batch2 + epi_batch3 + epi_batch4

df_Avian_N1 = pd.DataFrame({"Years":years, "full_sequence":full_sequences, "Stalk_seq": stalk_list, "Stalk_length":stalk_length, "epi":epi})

In [ ]:
df_Avian_N1 = df_Avian_N1[(df_Avian_N1["Years"] > 1989) & (df_Avian_N1["Stalk_length"] > 0)]

decades = []

for year in df_Avian_N1["Years"]:
    if 1990 <= year <= 1999:
        decades.append("1990-99")
    elif 2000 <= year <= 2009:
        decades.append("2000-09")
    elif 2010 <= year <= 2019:
        decades.append("2010-19")
    elif 2020 <= year <= 2023:
        decades.append("2020-23")

df_Avian_N1["Decade"] = decades
df_Avian_N1_90_99 = df_Avian_N1[df_Avian_N1["Decade"] == "1990-99"]
df_Avian_N1_00_09 = df_Avian_N1[df_Avian_N1["Decade"] == "2000-09"]
df_Avian_N1_10_19 = df_Avian_N1[df_Avian_N1["Decade"] == "2010-19"]
df_Avian_N1_20_23 = df_Avian_N1[df_Avian_N1["Decade"] == "2020-23"]

In [ ]:
sns.set(font_scale = 2)
sns.set_style("whitegrid")

fig, ax = plt.subplots(figsize=(10, 6))

sns.kdeplot(df_Avian_N1_90_99["Stalk_length"], fill=True, palette="ch:.35",color = "blue", label="1990-99",  alpha = 0.5)
sns.kdeplot(df_Avian_N1_00_09["Stalk_length"], fill=True, palette="ch:.35",color = "orange", label="2000-09",  alpha = 0.5)
sns.kdeplot(df_Avian_N1_10_19["Stalk_length"], fill=True, palette="ch:.35",color = "green", label="2010-19",  alpha = 0.5)
sns.kdeplot(df_Avian_N1_20_23["Stalk_length"], fill=True, palette="ch:.35",color = "red", label="2020-23",  alpha = 0.5)

plt.legend()
ax.legend(title = "Years", loc='upper center', bbox_to_anchor=(0.16, 1.03), ncol=1)
plt.xlabel('Stalk length', fontsize = 30, labelpad=10)
plt.ylabel('Density', fontsize = 30, labelpad=10)



plt.xlim(10,70)
plt.ylim(0,0.6)
plt.yticks(rotation=0, fontsize=28)
plt.xticks(rotation=0, fontsize=28)


plt.savefig('Avian_N1_stalk_length.tiff', dpi = 600, bbox_inches='tight')

**HA glycosylation**

In [ ]:
def get_HA_df(fasta_file):
  # Define the regex patterns to search for within the header lines
  pattern_epid = r'\|EPI_ISL_\d+\|'
  pattern_date = r'\d{4}-\d{2}-\d{2}'

  # Initialize lists to store identifiers, dates, and sequences
  identifiers = []
  dates = []
  sequences = []

  # Open the FASTA file for reading
  with open(fasta_file, "r") as fasta_file:
    current_header = ""
    current_sequence = ""
    for line in fasta_file:
        line = line.strip()
        # Check if the line is a header line
        if line.startswith('>'):
          # If we already have a header and sequence, process them
            if current_header:
              epi_match = re.search(pattern_epid, current_header)
              date_match = re.search(pattern_date, current_header)

              if epi_match and date_match:
                identifiers.append(epi_match.group(0))
                dates.append(date_match.group(0))
                sequences.append(current_sequence)

              current_sequence = ""

            current_header = line
        else:
            current_sequence += line

    # Process the last sequence
    if current_header:
        epi_match = re.search(pattern_epid, current_header)
        date_match = re.search(pattern_date, current_header)

        if epi_match and date_match:
            identifiers.append(epi_match.group(0))
            dates.append(date_match.group(0))
            sequences.append(current_sequence)
  # Create a DataFrame
  data = {'EPI_ID': identifiers, 'Date': dates, 'H5_Sequence': sequences}
  df = pd.DataFrame(data)

  # Identify the putative glycosylation sites
  gls = r'[Nn][A-Za-z][SsTt]'
  gls_no = []

  for i in df["H5_Sequence"]:
    matches = re.findall(gls,i)
    match_count = len(matches)
    df.loc[df["H5_Sequence"] == i, "#GLS"] = match_count
    gls_no.append(match_count)

  df["#GLS"] = gls_no

  return df

In [ ]:
# Avian stalk length - clade 2.3.4.4b
years_batch1,full_sequences_1, stalk_list_1, stalk_length_batch1, epi_batch1,_ =  get_stalk_length_year("N1-2.3.4.4b-batch1-msa.txt", "N1-2.3.4.4b-batch1-msa-date.txt",4000, r'G[-]*N[-]*I[-]*I[-]*S[-]*x[-]*x', r'V[-]*x[-]*x[-]*x[-]*T[-]*L', "I", 1, "x", 1)
years_batch2,full_sequences_2, stalk_list_2, stalk_length_batch2, epi_batch2,_ =  get_stalk_length_year("N1-2.3.4.4b-batch2-msa.txt", "N1-2.3.4.4b-batch2-msa-date.txt", 2643, r'G[-]*N[-]*I[-]*I[-]*S[-]*x[-]*x', r'V[-]*x[-]*x[-]*x[-]*T[-]*L', "I", 1, "x", 1)


years = years_batch1 + years_batch2
full_sequences = full_sequences_1 + full_sequences_2
stalk_list = stalk_list_1 + stalk_list_2
stalk_length = stalk_length_batch1  + stalk_length_batch2
epi_id = epi_batch1 + epi_batch2


df_Avian_N1_4b = pd.DataFrame({"Years":years, "full_sequence":full_sequences, "Stalk_seq": stalk_list, "NA_Stalk_length":stalk_length, "Epi_id": epi_id})
df_Avian_N1_4b_ls = df_Avian_N1_4b[40 < df_Avian_N1_4b["NA_Stalk_length"] < 70]
df_Avian_N1_4b_ss = df_Avian_N1_4b[20 < df_Avian_N1_4b["NA_Stalk_length"] < 41]

# Get EPI ID and then sort the H5
epi_ls = df_Avian_N1_4b_ls["Epi_id"].to_list()
epi_ss = df_Avian_N1_4b_ss["Epi_id"].to_list()

In [ ]:
# Avian H5 dataframe - clade 2.3.4.4b H5N1
df_H5_4b = get_HA_df("H5N1-H5-2.3.4.4b.fasta")
df_H5_4b_ls = df_H5_4b[df_H5_4b["EPI_ID"].isin(epi_ls)]
df_H5_4b_ss = df_H5_4b[df_H5_4b["EPI_ID"].isin(epi_ss)]

In [ ]:
# Avian stalk length - clade 2.3.2.1a and 1c
years,full_sequences, stalk_list, stalk_length, epi_id,_ =  get_stalk_length_year("H5N1-N1-2.3.2.1a,1c-msa.txt", "H5N1-N1-2.3.2.1a,1c-msa-date.txt",343, r'G[-]*N[-]*I[-]*I[-]*S[-]*x[-]*x', r'V[-]*x[-]*x[-]*x[-]*T[-]*L', "I", 1, "x", 1)

df_Avian_N1_1a_1c = pd.DataFrame({"Years":years, "full_sequence":full_sequences, "Stalk_seq": stalk_list, "NA_Stalk_length":stalk_length, "Epi_id": epi_id})
df_Avian_N1_1a_1c_ls = df_Avian_N1_1a_1c[40 < df_Avian_N1_1a_1c["NA_Stalk_length"] < 70]
df_Avian_N1_1a_1c_ss = df_Avian_N1_1a_1c[20 < df_Avian_N1_1a_1c["NA_Stalk_length"] < 41]

# GET EPI ID and then sort the H5
epi_ls = df_Avian_N1_1a_1c_ls["Epi_id"].to_list()
epi_ss = df_Avian_N1_1a_1c_ss["Epi_id"].to_list()

In [ ]:
# Avian H5 dataframe - clade 2.3.2.1a and 1c H5N1
df_H5_1a_1c = get_HA_df("H5N1-H5-2.3.2.1a, 1c.fasta")

df_H5_1a_1c_ls = df_H5_1a_1c[df_H5_1a_1c["EPI_ID"].isin(epi_ls)]
df_H5_1ac_ss = df_H5_1a_1c[df_H5_1a_1c["EPI_ID"].isin(epi_ss)]

In [ ]:
fig, ax = plt.subplots(figsize = (8,6))
sns.set_style("whitegrid")
sns.kdeplot(df_H5_4b_ls["#GLS"], fill=True, label = "2.3.4.4b H5N1-ls", color = "green", alpha = 0.6)
sns.kdeplot(df_H5_4b_ss["#GLS"], fill=True, label = "2.3.4.4b H5N1-ss", color = "orange", alpha = 0.8)
sns.kdeplot(df_H5_1a_1c_ls["#GLS"], fill=True, label = "2.3.2.1a H5N1-ss", color = "red", alpha = 0.9)
sns.kdeplot(df_H5_1ac_ss["#GLS"], fill=True, label = "2.3.2.1c H5N1-ss", color = "violet", alpha = 0.7)


plt.xlim(4,12)
ax.set_xticks(np.arange(4, 13, 1))

plt.ylim(0,6)

# Move the legend below the plot in one row
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.30), ncol=2)

# Set labels and title
plt.xlabel('No.of glycosylation sites')
plt.ylabel('Density')
#plt.title('H5N1 GLS coverage distribution 2018-2023')

matplotlib.rcParams.update({'font.size': 18})

# Save the plot
plt.savefig('H5N1_gls_18_23.tiff',  dpi=600,bbox_inches='tight')

# Show the plot
plt.show()

**Bubble plot**

In [ ]:
df = pd.read_csv("H5_2018_23.csv")

fig = px.scatter(df, x="#GLS", y="NA type",
                 size="Frequency", color="NA_Stalk_length", size_max=80, color_continuous_scale="Plasma")

fig.update_layout(
    font=dict(
        family="Arial",
        size=20,  # Set the font size here
        #color="RebeccaPurple"
    )
)
fig.update_layout(width=700, height=500)
fig.update_xaxes(range=[4, 10])
fig.update_xaxes(title="")
fig.update_yaxes(title="")

fig.update_traces(marker=dict(colorscale='Purpor'))

# Hide the color bar
fig.update_layout(coloraxis_showscale=False)

fig.write_image("H5_bubble_2018_23.pdf")
fig.show()
